In [ ]:
"""
amp_0.pth: 0.04988
amp_1.pth: 0.10548
amp_2.pth: 0.14452
amp_3.pth: 0.18068
amp_4.pth: 0.20834
amp_5.pth: 0.23518
amp_6.pth: 0.25766
amp_7.pth: 0.27846
amp_8.pth: 0.29652
amp_9.pth: 0.3141
amp_10: 32.882
amp_11: 34.244
amp_12: 35.138
amp_13: 36.166
amp_14: 37.16
amp_15: 37.574
amp_16.pth: 0.39168
amp_17.pth: 0.39838
amp_18.pth: 0.40566
amp_19.pth: 0.40992
"""

In [1]:
%load_ext autoreload
%autoreload 2

In [147]:
%matplotlib inline
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

from model import VisionTransformer
from data import imagenet_train_transform, imagenet_eval_transform, CLASS_NAME

In [189]:
state_dict = torch.load('/home/john/john/repaper_qad/vision_transformer.pytorch/logs/amp_12.pth')
print(state_dict.keys())

dict_keys(['model', 'optimizer'])


In [190]:
model = VisionTransformer()
model.load_state_dict(state_dict['model'])
model.cuda()

VisionTransformer(
  (patch_embedding): Conv2d(3, 768, kernel_size=(28, 28), stride=(28, 28))
  (encoder): Encoder(
    (blocks): Sequential(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (key_project): Linear(in_features=768, out_features=9216, bias=True)
          (query_project): Linear(in_features=768, out_features=9216, bias=True)
          (value_project): Linear(in_features=768, out_features=9216, bias=True)
          (final_project): Linear(in_features=9216, out_features=768, bias=True)
          (drop_layer): Dropout(p=0.1, inplace=False)
        )
        (drop_layer1): Dropout(p=0.1, inplace=False)
        (drop_layer2): Dropout(p=0.1, inplace=False)
        (linear): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): ReLU()
          (2): Dropout(p=0.1, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
        )
        (layer_norm1): LayerNorm((768,), eps=1e-05, ele

In [146]:
val = datasets.ImageFolder(
    root='/home/john/john/data/imagenet/val',
    transform=imagenet_eval_transform)

In [112]:
train = datasets.ImageFolder(
    root='/home/john/john/data/imagenet/train')

In [157]:
IDX = 0
model.eval()
with torch.no_grad():
    input_, label_ = val[IDX]
    input_ = input_.cuda().unsqueeze(0)
    output = model(input_)
    output = torch.softmax(output, dim=1)
    pred = torch.argmax(output, dim=1).item()
    print(f"Pred: {pred}, {CLASS_NAME[pred]}")
    print(f"Label: {label_}, {CLASS_NAME[label_]}")

Pred: 0, tench, Tinca tinca
Label: 0, tench, Tinca tinca


In [171]:
val_loader = DataLoader(val, batch_size=256, num_workers=16).__iter__()

In [191]:
val_loader = DataLoader(val, batch_size=256, num_workers=16)
model.eval()
total, correct = 0, 0
idx = 0
for input_, label_ in val_loader:
    with torch.no_grad():
        input_, label_ = input_.cuda(), label_.cuda()
        output = model(input_)
        output = torch.softmax(output, dim=1)
        pred = torch.argmax(output, dim=1)
        
        total += output.size(0)
        correct += (label_ == pred).sum().item()
    idx += 1

print(correct / total)

0.35138


In [197]:
result = []
for idx in range(16,20):
    file_name = f'amp_{idx}.pth'
    state_dict = torch.load(f'/home/john/john/repaper_qad/vision_transformer.pytorch/logs/{file_name}')
    model = VisionTransformer()
    model.load_state_dict(state_dict['model'])
    model.cuda()

    val_loader = DataLoader(val, batch_size=256, num_workers=16)
    model.eval()
    total, correct = 0, 0
    idx = 0
    for input_, label_ in val_loader:
        with torch.no_grad():
            input_, label_ = input_.cuda(), label_.cuda()
            output = model(input_)
            output = torch.softmax(output, dim=1)
            pred = torch.argmax(output, dim=1)

            total += output.size(0)
            correct += (label_ == pred).sum().item()
        idx += 1

    result.append(f'{file_name}: {correct / total}')

In [199]:
result

['amp_16.pth: 0.39168',
 'amp_17.pth: 0.39838',
 'amp_18.pth: 0.40566',
 'amp_19.pth: 0.40992']

In [8]:
datasets.imagenet.parse_val_archive(
    root='/home/john/john/data'
)

> /mnt/ai_filestore/home/john/miniconda3/envs/vision-transformer/lib/python3.7/site-packages/torchvision/datasets/imagenet.py(217)parse_val_archive()
    215     import pdb; pdb.set_trace()
    216 
--> 217     images = sorted([os.path.join(val_root, image) for image in os.listdir(val_root)])
    218 
    219     for wnid in set(wnids):



ipdb>  pp val_root


'/home/john/john/data/val'


ipdb>  c


In [8]:
from pathlib import Path

In [9]:
train_dir = Path('/home/john/john/data/imagenet/train')
val_dir = Path('/home/john/john/data/imagenet/val')

train_classes = [each.name for each in train_dir.glob('*')]
val_classes = [each.name for each in val_dir.glob('*')]

print(len(train_classes), len(val_classes))

1000 1000


In [11]:
len(set(val_classes).difference(set(train_classes)))

0